# Model

> Fill in a module description here

In [ ]:
#| default_exp model

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F
import torchvision
from torchvision.datasets import MNIST
from torchvision import datasets, transforms
import lightning.pytorch as lightning
from lightning.pytorch.core import LightningModule

class LightningCifarClassifier(LightningModule):

  def __init__(self, arch="resnet18"):
    super(LightningCifarClassifier, self).__init__()
    self.save_hyperparameters()
    self.model = nn.Sequential(
        # Conv2d( in_channels, out_channels, kernel_size) B,3,32,32
        nn.Conv2d(3, 32, 3, stride= 2, padding=1),  nn.BatchNorm2d(32), nn.Tanh(), # B,32,16,16
        nn.Conv2d(32, 64, 3, stride= 2, padding=1), nn.BatchNorm2d(64), nn.Tanh(), # B,32,8,8
        nn.Conv2d(64, 128, 3, stride= 2, padding=1), nn.BatchNorm2d(128), nn.Tanh(), # B,32,4,4
        nn.Flatten(),
        nn.Linear(4*4*128, 64), nn.BatchNorm1d(64), nn.Tanh(),
        nn.Linear(64, 10)
    )

  def forward(self, x):
      logits = self.model(x)
      return logits

  def training_step(self, train_batch, batch_idx):
      x, y = train_batch
      logits = self.forward(x)   # we already defined forward and loss in the lightning module. We'll show the full code next
      loss = F.cross_entropy(logits,y)
      self.log('train_loss',loss)

      return {'loss': loss}

  def validation_step(self, val_batch, batch_idx):
      x, y = val_batch
      logits = self.forward(x)
      loss = F.cross_entropy(logits,y)
      self.log('val_loss', loss)
      self.log('avg_val_loss', loss, on_epoch=True)

      return {'val_loss': loss}


  def prepare_data(self):
    torchvision.datasets.CIFAR10(root='./data', download=True, train=True)
    torchvision.datasets.CIFAR10(root='./data', download=True, train=False)

  def train_dataloader(self):
    transform = transforms.Compose(
    [
      transforms.RandomHorizontalFlip(0.5),
      transforms.ColorJitter(0.2,0.2,0.2),
      transforms.ToTensor(),
      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
    ])
    cifar10 = torchvision.datasets.CIFAR10(root='./data', download=False, train=True, transform=transform)

    dataloader = DataLoader(cifar10, batch_size=32, shuffle=True, num_workers=2)
    return dataloader

  def val_dataloader(self):
    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
        ])
    cifar10_test = torchvision.datasets.CIFAR10(root='./data', download=False, train=False, transform=test_transform)
    dataloader_test = DataLoader(cifar10_test, batch_size=32, shuffle=False, num_workers=2)
    return dataloader_test

  def configure_optimizers(self):
    # the lightningModule HAS the parameters (remember that we had the __init__ and forward method but we're just not showing it here)

    optimizer = torch.optim.Adam(self.parameters(),0.001)
    # optimizer =  torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.00001)
    return optimizer


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()